# Output Parser로 LLM 응답 파싱하기

이 노트북에서는 **CommaSeparatedListOutputParser**를 사용하여 LLM 응답을 리스트로 변환하는 방법을 알아봅니다.

## Output Parser란?

LLM의 텍스트 응답을 **원하는 형식으로 변환**하는 도구입니다.

```
LLM 응답 (문자열)  →  Output Parser  →  Python 객체 (리스트, 딕셔너리 등)
```

## LangChain 주요 Output Parser

| Parser | 입력 예시 | 출력 타입 |
|--------|----------|----------|
| `StrOutputParser` | "안녕하세요" | `str` |
| `CommaSeparatedListOutputParser` | "a, b, c" | `['a', 'b', 'c']` |
| `JsonOutputParser` | '{"key": "value"}' | `dict` |
| `PydanticOutputParser` | JSON 문자열 | Pydantic 객체 |

## CommaSeparatedListOutputParser의 장점

1. **간단한 사용법**: 쉼표로 구분된 텍스트를 리스트로 변환
2. **공백 자동 처리**: `"a, b, c"` → `['a', 'b', 'c']` (앞뒤 공백 제거)
3. **LCEL 호환**: 체인에 `| parser` 형태로 연결 가능

---

# 1. 패키지 설치

In [1]:
!pip install -q langchain-core

# 2. CommaSeparatedListOutputParser 기본 사용법

**코드 설명:**

### Parser 생성 및 실행
```python
parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")
```

- `CommaSeparatedListOutputParser()` - 파서 인스턴스 생성
- `parser.invoke(문자열)` - 쉼표로 구분된 문자열을 리스트로 변환
- 각 항목의 **앞뒤 공백이 자동으로 제거**됨

### 변환 과정
```
입력: "apple, banana, cherry"
      ↓ parser.invoke()
출력: ['apple', 'banana', 'cherry']
```

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")
print(items)

['apple', 'banana', 'cherry']


# 3. LLM과 함께 사용하기 (LCEL 체인)

실제로는 LLM 응답을 파싱하는 데 사용합니다. 아래는 Ollama와 함께 사용하는 예시입니다.

```python
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 컴포넌트 생성
parser = CommaSeparatedListOutputParser()
model = ChatOllama(model='llama3.2')
prompt = ChatPromptTemplate.from_messages([
    ('system', '요청에 맞는 항목들을 쉼표로 구분하여 나열하세요.'),
    ('human', '{query}')
])

# LCEL 체인 구성
chain = prompt | model | parser

# 실행
result = chain.invoke({'query': '한국의 대표적인 음식 5가지'})
print(result)  # ['김치', '비빔밥', '불고기', '떡볶이', '삼겹살']
```

## 체인 흐름

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   입력       │ ──▶ │   Prompt    │ ──▶ │    Model    │ ──▶ │   Parser    │
│  {query}    │     │  Template   │     │  (LLM 호출)  │     │ (리스트변환)   │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                                                                   │
                                                                   ▼
                                                         ['김치', '비빔밥', ...]
```

---

## format_instructions 활용

Parser가 제공하는 **형식 지시사항**을 프롬프트에 포함시킬 수 있습니다:

```python
parser = CommaSeparatedListOutputParser()

# 형식 지시사항 확인
print(parser.get_format_instructions())

# 출력: "Your response should be a list of comma separated values, eg: `foo, bar, baz`"

# 프롬프트에 포함
prompt = ChatPromptTemplate.from_messages([
    ('system', '다음 형식으로 답변하세요: {format_instructions}'),
    ('human', '{query}')
])

chain = prompt | model | parser
result = chain.invoke({
    'query': '프로그래밍 언어 3가지',
    'format_instructions': parser.get_format_instructions()
})
```

## 다른 Output Parser 예시

### StrOutputParser (가장 기본)
```python
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser  # AIMessage → str
```

### JsonOutputParser
```python
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
result = parser.invoke('{"name": "홍길동", "age": 30}')
# {'name': '홍길동', 'age': 30}
```

## 활용 팁

1. **리스트 응답 필요 시**: `CommaSeparatedListOutputParser` 사용
2. **단순 문자열 추출**: `StrOutputParser` 사용 (AIMessage → str)
3. **구조화된 데이터**: `JsonOutputParser` 또는 `with_structured_output()` 사용
4. **프롬프트 개선**: `get_format_instructions()`로 형식 지시 포함